In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
%matplotlib inline

In [5]:
url = 'https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/6_gradient_descent/insurance_data.csv?_sm_au_=iVV1jfpWMDF8j706kG7NjKsqsN873'
df = pd.read_csv(url)
df.head(2)

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [7]:
# escalar las edades
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation="sigmoid", kernel_initializer="ones", bias_initializer="zeros")
])

In [11]:
model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=["accuracy"])
model.fit(X_train_scaled, y_train, epochs=5000)

Se han truncado las últimas 5000 líneas del flujo de salida.
1/1 [==============================] - 0s 19ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 24ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 18ms/step - loss: 0.5318 - accuracy: 0.7273
Epoch 2510/5000
1/1 [================

In [12]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 204ms/step - loss: 0.3550 - accuracy: 1.0000


[0.3549776077270508, 1.0]

In [13]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 148ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)

In [14]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [15]:
# Ahora para saber los pesos
# de la funcion
coef, intercept = model.get_weights()
print("coef", coef)
print("intercept", intercept)

coef [[5.060863 ]
 [1.4086521]]
intercept [-2.913703]


# Neural network from scratch

In [16]:
import math
def sigmoid(x):
  return 1 / (1+ math.exp(-x))

In [19]:
def prediction(age, affordibility, coef, intercept):
  w_sum = coef[0]*age + coef[1] * affordibility + intercept
  return sigmoid(w_sum)

In [20]:
prediction(.47, 1, coef, intercept)
# esto es lo que regreso la rted neuronal
# pero ya le estamos dando los coefs e intercept

<ipython-input-16-76840bc37b87>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1+ math.exp(-x))


0.705484819775958

In [21]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [22]:
def sigmoid_numpy(X):
  # para vectores
  return 1/ (1+np.exp(-X))

In [48]:
def gradient_descent(age, affordibility, y_true, epochs, loss_thresh):
  # inicializando
  w1 = w2 = 1
  bias = 0
  rate = 0.1
  n = len(age)
  for i in range(epochs):

    w_sum = w1 * age + w2 * affordibility + bias
    y_predicted = sigmoid_numpy(w_sum)
    loss = log_loss(y_true, y_predicted)
    acc = sklearn.metrics.accuracy_score(y_true, np.round(y_predicted))


    # actualizando pesos con derivadas
    w1_d = np.mean(age*(y_predicted - y_true))
    w2_d = np.mean(affordibility*(y_predicted - y_true))
    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - rate * w1_d
    w2 = w2 - rate * w2_d
    bias = bias - rate * bias_d

    print(f"loss = {loss} in epoch {i+1} with accuracy = {acc}")

    if loss <= loss_thresh:
      break

  return w1,w2, bias






In [49]:
w1,w2, bias = gradient_descent(X_train_scaled["age"], X_train_scaled["affordibility"], y_train, 5000, 0.4631)

loss = 0.7113403233723417 in epoch 1 with accuracy = 0.5
loss = 0.7049637757302932 in epoch 2 with accuracy = 0.5
loss = 0.6989316069171762 in epoch 3 with accuracy = 0.5
loss = 0.6932296440186102 in epoch 4 with accuracy = 0.5
loss = 0.6878437615034732 in epoch 5 with accuracy = 0.5
loss = 0.6827599423171782 in epoch 6 with accuracy = 0.5
loss = 0.6779643338517055 in epoch 7 with accuracy = 0.5
loss = 0.6734432985751392 in epoch 8 with accuracy = 0.5
loss = 0.6691834592087409 in epoch 9 with accuracy = 0.5
loss = 0.6651717384356765 in epoch 10 with accuracy = 0.5454545454545454
loss = 0.6613953932110402 in epoch 11 with accuracy = 0.5454545454545454
loss = 0.6578420438168681 in epoch 12 with accuracy = 0.5454545454545454
loss = 0.6544996978680806 in epoch 13 with accuracy = 0.5454545454545454
loss = 0.6513567695257338 in epoch 14 with accuracy = 0.5454545454545454
loss = 0.6484020942130568 in epoch 15 with accuracy = 0.5909090909090909
loss = 0.6456249391581439 in epoch 16 with accura

In [51]:
print("coef with programmed", w1,w2)
print("intercept with programmed", bias)

coef with programmed 5.036910425410707 1.4564630137800754
intercept with programmed -2.953436064358123


In [50]:
print("coef with keras", coef)
print("intercept with keras", intercept)

coef with keras [[5.060863 ]
 [1.4086521]]
intercept with keras [-2.913703]
